In [130]:
# Import Dependencies
import pandas as pd
import datetime as dt

In [131]:
# Read in Monthly Sales Price Data
price = pd.read_excel('Data/Med:Avg Sales Price.xls', sheet_name='Price ')

In [132]:
# Clean up DataFrame
price.columns = price.iloc[1]
price = price.iloc[2:]

In [133]:
# Remove timestamp from Period column
price['Period'] = pd.to_datetime(price['Period'], errors='coerce')

In [134]:
# Insert Median prices into empty Average values
price['Average'].loc[2:145] = price['Median '].loc[2:145]

/var/folders/zq/0hmyg29903n1xkl833j8x3h00000gn/T/ipykernel_19094/3224460076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Average'].loc[2:145] = price['Median '].loc[2:145]


In [135]:
# Drop Median column
price.drop(columns='Median ', inplace=True)

In [136]:
# Set Period as index
price.set_index('Period', inplace=True)
price = price.rename_axis('Period', axis='columns')
price.index.name = None

In [138]:
price

Period,Average
1963-01-01,17200
1963-02-01,17700
1963-03-01,18200
1963-04-01,18200
1963-05-01,17500
...,...
2022-02-01,513900
2022-03-01,522500
2022-04-01,570300
NaT,NaN
